In [1]:
%load_ext autoreload
%autoreload 2

Running this example requires [pyLODE](https://github.com/ceriottm/lode).

In [2]:
import numpy as np
import ase.io

from utils.pylode import PyLODESphericalExpansion
import utils.io
from utils.models.soap import compute_power_spectrum

In [3]:
n_frames = 3
frames = ase.io.read('data/BaTiO3_Training_set.xyz', f':{n_frames}')

In [4]:
# LODE calculations can take quite a while...
# Therefore, it is useful to store results and only recalculate them if 
# needed!

recalc = True

In [5]:
# Define hyperparameters
lode_hypers = dict(
    smearing=3,
    max_radial=2,
    max_angular=3,
    cutoff_radius=4.5,
    potential_exponent=1,
    radial_basis='GTO',
    compute_gradients=True,
)

calculator = PyLODESphericalExpansion(lode_hypers)

fname = "precomputed.h5"
if recalc:
    descriptor = calculator.compute(frames)
    utils.io.write(fname, descriptor)
else:
    descriptor = utils.io.read(fname)

The Lode descripor can also be precomputed using the `precompute_lode.py` from 
the script directory. 

You can either precompute the descriptor from the python 
interface

In [6]:
from scripts.precompute_lode import precompute_lode

In [7]:
descriptor = precompute_lode(frames, lode_hypers)

Or from the command line with

```bash
scripts/precompute_lode.py -input_file "data/BaTiO3_Training_set.xyz" \
                           -index ":10" \
                           -max_radial 2 \
                           -max_angular 0 \
                           -cutoff_radius 4.5 \
                           -smearing 3 \
                           -radial_basis "GTO" \
                           -subtract_center_contribution \
                           -compute_gradients \
                           -outfile "precomputed_lode"
```

The exact syntax is given on the help page

```bash
scripts/precompute_lode.py -h
```

# Application: Compute power spectrum

### Example usage: Multiscale invariants (rho x V) combining short and long range

In [8]:
# Define hyperparameters for short range (SR) densities
hypers_sr = dict(
    smearing=3,
    max_radial=2,
    max_angular=1,
    cutoff_radius=4.5,
    potential_exponent=0, # 0 here means Gaussian/SR densities
    radial_basis='GTO',
    compute_gradients=True,
)
calculator_sr = PyLODESphericalExpansion(hypers_sr)
descriptor_sr = calculator_sr.compute(frames)

# Define hyperparameters for long range (LR) densities
hypers_lr = dict(
    smearing=3,
    max_radial=1,
    max_angular=1,
    cutoff_radius=4.5,
    potential_exponent=1, # 1 here means Coulombic/LR densities
    radial_basis='monomial',
    compute_gradients=True,
)
calculator_lr = PyLODESphericalExpansion(hypers_lr)
descriptor_lr = calculator_sr.compute(frames)

In [9]:
# Generate SR x SR (rho x rho) invariants
invariants_srsr = compute_power_spectrum(descriptor_sr)
# Generate LR x LR (V x V) invariants
invariants_lrlr = compute_power_spectrum(descriptor_lr)
# Generate SR x LR (rho x V)
invariants_multiscale = compute_power_spectrum(descriptor_sr, descriptor_lr)

### Test that the coefficients obtained from the general method agree with those from the specialized version

In [15]:
invariants_srsr_2 = compute_power_spectrum(descriptor_sr, descriptor_sr)
for sparse in invariants_srsr.sparse:
    # Get block from original code
    block_ref = invariants_srsr.block(center_species=sparse[0],
                         neighbor_species_1=sparse[1],
                         neighbor_species_2=sparse[2])

    # Compare this to invariants obtained from generalization
    block_1 = invariants_srsr_2.block(center_species=sparse[0],
                                      neighbor_species_1=sparse[1],
                                      neighbor_species_2=sparse[2])

    # Also compare with the invariants with swapped neighbor species
    block_2 = invariants_srsr_2.block(center_species=sparse[0],
                                      neighbor_species_1=sparse[2],
                                      neighbor_species_2=sparse[1])

    assert block_ref.values.shape == block_1.values.shape
    assert block_ref.values.shape == block_2.values.shape
    assert np.linalg.norm(block_ref.values - block_1.values) < 1e-15
    assert np.linalg.norm(block_ref.values - block_2.values) < 1e-15